In [13]:
import cv2
import kpl

ImportError: No module named 'kpl'

In [2]:
# 单应性估计

### 0.Read in an image file, errors out if we can't find the file

In [22]:
def readImage(filename):
    img = cv2.imread(filename, 0)
    if img is None:
        print('Invalid image:' + filename)
        return None
    else:
        print('Image successfully read...')
        return img


### 1.Runs sift algorithm to find features/被文本检测的点替代

In [23]:
def findFeatures(img):
    print("Finding Features...")
    sift = cv2.SIFT()
    keypoints, descriptors = sift.detectAndCompute(img, None)

    img = cv2.drawKeypoints(img, keypoints)
    cv2.imwrite('sift_keypoints.png', img)

    return keypoints, descriptors

### 2.Matches features given a list of keypoints, descriptors, and images / /被文本检测的点替代

In [24]:
def matchFeatures(kp1, kp2, desc1, desc2, img1, img2):
    print("Matching Features...")
    matcher = cv2.BFMatcher(cv2.NORM_L2, True)
    matches = matcher.match(desc1, desc2)
    matchImg = drawMatches(img1,kp1,img2,kp2,matches)
    cv2.imwrite('Matches.png', matchImg)
    return matches

### 3.Computers a homography from 4-correspondences /

In [25]:
def calculateHomography(correspondences):
    #loop through correspondences and create assemble matrix
    aList = []
    for corr in correspondences:
        p1 = np.matrix([corr.item(0), corr.item(1), 1])
        p2 = np.matrix([corr.item(2), corr.item(3), 1])

        a2 = [0, 0, 0, -p2.item(2) * p1.item(0), -p2.item(2) * p1.item(1), -p2.item(2) * p1.item(2),
              p2.item(1) * p1.item(0), p2.item(1) * p1.item(1), p2.item(1) * p1.item(2)]
        a1 = [-p2.item(2) * p1.item(0), -p2.item(2) * p1.item(1), -p2.item(2) * p1.item(2), 0, 0, 0,
              p2.item(0) * p1.item(0), p2.item(0) * p1.item(1), p2.item(0) * p1.item(2)]
        aList.append(a1)
        aList.append(a2)

    matrixA = np.matrix(aList)

    #svd composition
    u, s, v = np.linalg.svd(matrixA)

    #reshape the min singular value into a 3 by 3 matrix
    h = np.reshape(v[8], (3, 3))

    #normalize and now we have h
    h = (1/h.item(8)) * h
    return h


### 4.Calculate the geometric distance between estimated points and original points

In [26]:
def geometricDistance(correspondence, h):

    p1 = np.transpose(np.matrix([correspondence[0].item(0), correspondence[0].item(1), 1]))
    estimatep2 = np.dot(h, p1)
    estimatep2 = (1/estimatep2.item(2))*estimatep2

    p2 = np.transpose(np.matrix([correspondence[0].item(2), correspondence[0].item(3), 1]))
    error = p2 - estimatep2
    return np.linalg.norm(error)


### 5.Runs through ransac algorithm, creating homographies from random correspondences

In [27]:
def ransac(corr, thresh):
    maxInliers = []
    finalH = None
    for i in range(1000):
        #find 4 random points to calculate a homography
        corr1 = corr[random.randrange(0, len(corr))]
        corr2 = corr[random.randrange(0, len(corr))]
        randomFour = np.vstack((corr1, corr2))
        corr3 = corr[random.randrange(0, len(corr))]
        randomFour = np.vstack((randomFour, corr3))
        corr4 = corr[random.randrange(0, len(corr))]
        randomFour = np.vstack((randomFour, corr4))

        #call the homography function on those points
        h = calculateHomography(randomFour)
        inliers = []

        for i in range(len(corr)):
            d = geometricDistance(corr[i], h)
            if d < 5:
                inliers.append(corr[i])

        if len(inliers) > len(maxInliers):
            maxInliers = inliers
            finalH = h
        print ("Corr size: ", len(corr), " NumInliers: ", len(inliers), "Max inliers: ", len(maxInliers))

        if len(maxInliers) > (len(corr)*thresh):
            break
    return finalH, maxInliers


### 6. estimate Homography matrix 

In [31]:
estimation_thresh = 0.6
print ("Estimation Threshold: ", estimation_thresh)
if estimation_thresh is None:
    estimation_thresh = 0.60

img_name = ["C:\Users\Tianh\Desktop\毕设\2-homography\pic1.jpg", "C:\Users\Tianh\Desktop\毕设\2-homography\pic2.jpg"]
img1name = str(img_name[0])
img2name = str(img_name[1])
print("Image 1 Name: " + img1name)
print("Image 2 Name: " + img2name)

#query image
img1 = readImage(img_name[0])
#train image
img2 = readImage(img_name[1])

#find features and keypoints
correspondenceList = []
if img1 is not None and img2 is not None:
    kp1, desc1 = findFeatures(img1)
    kp2, desc2 = findFeatures(img2)
    print ("Found keypoints in " + img1name + ": " + str(len(kp1)))
    print ("Found keypoints in " + img2name + ": " + str(len(kp2)))
    keypoints = [kp1,kp2]
    

matches = matchFeatures(kp1, kp2, desc1, desc2, img1, img2)
for match in matches:
    (x1, y1) = keypoints[0][match.queryIdx].pt
    (x2, y2) = keypoints[1][match.trainIdx].pt
    correspondenceList.append([x1, y1, x2, y2])

corrs = np.matrix(correspondenceList)

#run ransac algorithm
finalH, inliers = ransac(corrs, estimation_thresh)
print ("Final homography: ", finalH)
print ("Final inliers count: ", len(inliers))

matchImg = drawMatches(img1,kp1,img2,kp2,matches,inliers)
cv2.imwrite('InlierMatches.png', matchImg)

f = open('homography.txt', 'w')
f.write("Final homography: \n" + str(finalH)+"\n")
f.write("Final inliers count: " + str(len(inliers)))
f.close()

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (<ipython-input-31-9ac905f778b4>, line 6)

### 7.decompose Homography matrix to get camera parameters

In [29]:
# 编译C代码
#[r,u,k,n] = TransformDecompose(H);

In [4]:
H = cv2.detail_HomographyBasedEstimator()

AttributeError: module 'cv2' has no attribute 'detail_HomographyBasedEstimator'

In [7]:
# 相机标定

In [ ]:
retval, cameraMatrix, distCoeffs, rvecs, tvecs = cv2.calibrateCamera() 